<a href="https://colab.research.google.com/github/oscarmejias/deeplearning/blob/master/Stock_Market_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importamos las librerias**

In [3]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


##**Construimos la red neuronal de la AI del Trader**

In [0]:
class AI_Trader():
  def __init__(self, state_size, action_space = 3, model_name = "AITrader"): #Manten, Compra, Vende
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen= 2000)
    self.inventory = []
    self.model_name = model_name

    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995

    self.model = self.model_builder()

  def model_builder(self):

    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Dense(units= 32, activation= "relu", input_dim= self.state_size))

    model.add(tf.keras.layers.Dense(units= 64, activation= "relu"))

    model.add(tf.keras.layers.Dense(units= 128, activation= "relu"))

    model.add(tf.keras.models.Dense(units= self.action_space, activation= "linear"))

    model.compile(loss= "mse", optimizers= tf.keras.optimizers.Adam(learning_rate= 0.001))

    return model

  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)

    actions = self.model.predict(state)
    return np.argmax(actions[0])

  def batch_train(self, batch_size):

    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      print(len(self.memory) - batch_size + 1, len(self.memory)) # Curiosidad, borrar despues
      batch.append(self.memory[i])

    for state, action, reward, next_action, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        print("esto es todo lo que arroja una prediccion: {}".format(model.predict(next_state))) # Analizo el formato de lo que es una prediccion del modelo
        print("este es el valor maximo de las predicciones: {}".format(np.amax(model.predict(next_state)[0]))) # Observo la prediccion mas alta del modelo

      target = self.model.predict(state)
      target[0][action] = reward

      print("Estado: {} -- Objetivo: {}".format(state, target)) # Imprimo el state y target para familiarizarme con el algoritmo
      self.model.fit(state, target, epochs= 1, verbose= 0)

    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay


##**Pre procesado del dataset**

**Definimos funciones adicionales**

In [0]:
#Defino la funcion sigmoide
def sigmoide(x):
  return 1/ (1 + math.exp(-x))

In [0]:
#Defino la funcion del formato de precios
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [0]:
#Defino la funcion que carga el Dataset
def dataset_loader(stock_name):

  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")

  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1].split()[0])

  close = dataset["Close"]

  return close

###**Creador de estados**


In [0]:
def state_creator(data, timestep, window_size):

  starting_id = timestep - window_size + 1

  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])

  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))

  return np.array([state])

###**Cargando una divisa del mercado**

In [0]:
stock_name = "UGAZ"
data = dataset_loader(stock_name)